### Importing Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

2024-07-10 18:11:53.398707: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 18:11:53.555129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 18:11:53.631954: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 18:11:53.632668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 18:11:53.761423: I tensorflow/core/platform/cpu_feature_gua

In [2]:
import pickle

def load_set(directory):
    try:
        with open(f"{directory}/v_texts.pkl", "rb") as fp:
            processed_texts = pickle.load(fp)
        
        with open(f"{directory}/labels.pkl", "rb") as fp:
            labels = pickle.load(fp)
    
    except:
        print(f'{directory} files not found. Please run the preprocess.ipynb before!')
    
    return processed_texts, labels

In [3]:
vectorized_texts, labels = load_set('train')
vectorized_val_texts, val_labels = load_set('val')
# processed_test_texts, test_labels = load_set('test')

In [5]:
len(vectorized_texts)

1000

### Creating the Model

#### Creating the model

In [34]:
class RaggedToDenseLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.ragged.stack(inputs, axis=0).to_tensor()

In [36]:
Sequential = tf.keras.models.Sequential

model = Sequential([
    tf.keras.layers.InputLayer(shape=(None,), dtype=tf.float32, ragged=True),
    RaggedToDenseLayer(),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(20)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

ValueError: Shapes used to initialize variables must be fully-defined (no `None` dimensions). Received: shape=(None, 80) for variable path='sequential_7/bidirectional_14/forward_lstm_14/lstm_cell/kernel'

In [22]:
h = model.fit(
    vectorized_texts, labels[:1000],
    validation_data=(vectorized_val_texts, val_labels),
    epochs=20,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ]
)

ValueError: Unrecognized data type: x=[Ragged(data=array([[ 0.25172982, -0.66466725,  0.06614111, ..., -0.6582263 ,
        -0.5213741 , -0.35608736],
       [ 0.5862261 , -0.84882635, -0.1899197 , ...,  1.6984494 ,
         0.6500878 , -0.06584859],
       [ 1.1921104 , -1.3467414 ,  0.1503521 , ...,  0.4264503 ,
         0.37842724, -0.9911235 ],
       ...,
       [ 0.35180095, -0.49391666, -0.36013862, ..., -0.8098457 ,
        -0.17668544, -0.48665687],
       [ 0.7250637 , -0.52557   , -0.5982763 , ..., -0.32575646,
         0.1170694 , -0.78299415],
       [ 0.56703717, -1.6354729 , -1.1350284 , ...,  0.5251709 ,
         0.3174584 , -0.43598813]], dtype=float32), lengths=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1], dtype=int32), data_shape=(-1, 768), starts_ends=None), Ragged(data=array([[ 0.3863544 ,  0.35048455,  0.43622798, ...,  1.0943649 ,
        -1.0303475 , -1.1168872 ],
       [ 0.20431203,  

### Evaluating the Model

In [ ]:
model.summary()

In [ ]:
vectorized_test_texts = vectorize_layer(processed_test_texts)

In [ ]:
_ = model.evaluate(vectorized_test_texts, test_labels)

In [ ]:
y_pred = model.predict(vectorized_test_texts)
y_pred = (y_pred > 0.5).astype(int)

y_pred = np.array(list(map(lambda x: x[0], y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()

fpr = fp / (fp + tn)
print(f"False Positive Rate: {fpr:.4f}")

fnr = fn / (fn + tp)
print(f"False Negative Rate: {fnr:.4f}")

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(test_labels, y_pred, normalize="true",
                                        values_format=".0%")
plt.show()